# ARIMA Grid Search

In [1]:
# 각 업종별 최적 hyperparameter와 RMSE 계산 함수
import itertools
def gridARIMA(train_series, test_series): # 특정 업종명의 날짜별로 수합 된 판매건수의 훈련 & 검증셋을 각각 받음
  # 실험용 pdq (무시)
  # p = q = d = range(0, 2)
  # 실제 pdq 값: p와 q는 0-7, d는 0-1
  p = q = range(0, 8)
  d = range(0, 2)
  
  # 최적의 hyperparameter와 RMSE를 계산 후 넣을 dictionary
  rmse = {}

  # pdq의 모든 조합 만들기
  pdq = list(itertools.product(p, d, q))

  # trend에 들어갈 값
  trend = ['c', 'nc']

  for param in pdq: # 각 pdq 조합별
    for item in trend: # 각 trend값 마다
      try:
        # 모델 구축
        model = ARIMA(train_series, order = param)
        model_fit = model.fit(trend = item, full_output = True, disp = 1)
        output = model_fit.forecast(steps = 31) # 예측값

        # 검증셋에 빈 날짜가 있을경우 RMSE 계산을 할 수 없으니, 예측값으로 채운 후 날짜로 정렬
        if len(pd.date_range('20200515', '20200614')[~pd.date_range('20200515', '20200614').isin(test_series.index)]):
          for date in pd.date_range('20200515', '20200614')[~pd.date_range('20200515', '20200614').isin(test_series.index)]:
            test_series.loc[date] = output[0][pd.date_range('20200515', '20200614').get_loc(date)]
        test_series = test_series.sort_index()
        
        # RMSE 계산 (key는 order와 trend에 들어가는 값 둘 다 가져오기 위해 같은 문자열에 넣고 추후 split으로 각각 구함)
        rmse[f'{param}|{item}'] = np.sqrt(mean_squared_error(test_series, output[0]))
      
      # 에러가 나면 멈추지 않고 다음 순서로 넘어감
      except: 
        continue
  # RMSE가 제일 낮은 조합 찾기
  key = min(rmse, key = rmse.get)
  # order와 trend hyperparameter로 나누기
  key_split = key.split('|')
  # order이 문자열로 되어있어 모델에 넣을 수 없으니, 넣을 수 있는 형식으로 변환
  key_split[0] = eval(key_split[0])
  # 업종명의 최적 hyperparameter (key_split)과 그 조합의 RMSE값을 리스트에 담아 반환
  return [key_split, rmse[key]]

# Applying Grid Search to get Predictions

In [ ]:
# 30일주기 제외 모든 업종명의 예측값과 RMSE 담기
def ARIMA_by_ind(df): # 전체 데이터 받음
  # 제외할 업종명
  day30 = ['생명보험', '소프트웨어', '가전제품', '전자상거래상품권전문판매', '정수기', '정기간행물', 
           '대형할인점', '통신기기', '위성방송', '통신판매', '전자상거래상품권', '학습지교육']
  
  # 결과를 담을 dictionary
  pred = {}
  
  # 모든 업종명
  names = df['업종명'].unique()
  
  # 훈련 & 검증셋 날짜기준 분할
  train = df[~df['날짜'].isin(pd.date_range('20200515', '20200614'))]
  test = df[df['날짜'].isin(pd.date_range('20200515', '20200614'))]
  
  for name in names: # 각 업종명별
    if name not in day30: # 제외하는 목록에 있지 않다면
      # 업종명의 날짜별 판매건수를 훈련 & 검증셋 내에서 수합
      train_ind = train[train['업종명'] == name].groupby('날짜')['판매건수'].sum()
      test_ind = test[test['업종명'] == name].groupby('날짜')['판매건수'].sum()
      
      # 최적의 hyperparameter와 
      best_param = gridARIMA(train_ind, test_ind)

      # 모델링
      model = ARIMA(train_ind, order = best_param[0][0])
      model_fit = model.fit(trend = best_param[0][1], full_output = True, disp = 1)
      
      # 예측값
      output = model_fit.forecast(steps = 31)
      
      # 결과를 담을 딕셔너리 안에 key를 업종명, value를 예측값과 RMSE가 담긴 리스트로 지정
      pred[name] = [output[0], best_param[1]]
  # 모든 업종명 계산 후 반환
  return pred